<a href="https://colab.research.google.com/github/cameronv10/Project-2/blob/main/Project_1_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

def stocks(ticker=True):
  url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol='+ticker+'&apikey=6JKXZSE93BPMCPCY'
  r = requests.get(url)
  data = r.json()
  stocks=pd.DataFrame.from_dict(data['Monthly Time Series'], orient='index')
  stocks.rename(columns={'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume': 'volume'}, inplace=True)
  stocks.reset_index(inplace=True)
  stocks=stocks.rename(columns = {'index':'date'})
  stocks['ticker']= ticker
  return stocks 

In [2]:
stocks(ticker='IBM')

,date,open,high,low,close,volume,ticker
0,2022-05-05,133.0000,137.8700,130.8900,135.9200,19773190,IBM
1,2022-04-29,129.6600,141.8800,124.9100,132.2100,107525264,IBM
2,2022-03-31,122.6700,133.0800,120.7000,130.0200,96447210,IBM
3,2022-02-28,133.7600,138.8200,118.8100,122.5100,98492968,IBM
4,2022-01-31,134.0700,142.2000,124.1930,133.5700,147238382,IBM
...,...,...,...,...,...,...,...
265,2000-04-28,120.0000,128.0000,101.2500,111.5000,168464800,IBM
266,2000-03-31,102.0000,128.2500,99.5000,118.3700,194329000,IBM
267,2000-02-29,112.3700,119.7500,100.9400,102.7500,133524400,IBM
268,2000-01-31,112.4400,124.7500,109.6200,112.2500,175259600,IBM


In [6]:
def quarter(month=True):
  temp=month/3
  if temp<=1:
    return 1
  elif temp>1 and temp<=2:
    return 2
  elif temp>2 and temp<=3:
    return 3
  else:
    return 4

In [7]:
import requests
import pandas as pd
ticker='IBM'

def Stocks(ticker=True):
  url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='+ticker+'&apikey=6JKXZSE93BPMCPCY'
  r = requests.get(url)
  data = r.json()
  stock=pd.DataFrame.from_dict(data['Monthly Adjusted Time Series']).T
  stock.reset_index(inplace=True)
  stock.rename(columns={'index':'date'},inplace=True)
  stock['date']=pd.to_datetime(stock['date'])
  stock['year']=stock['date'].dt.year
  stock['month']=stock['date'].dt.month
  columns=stock.columns
  for i in columns[1:8]:
    stock[i]= stock[i].astype(float)
  stock['quarter']=stock.apply(lambda x: quarter(x['month']),axis=1)
  stock=stock.groupby(['year','quarter'])['1. open', '7. dividend amount'].mean().reset_index()
  stock.rename(columns={'1. open':'open', '7. dividend amount':'dividend'}, inplace=True)
  stock['ticker']=ticker
  return stock

A=Stocks(ticker='A')
IBM=Stocks(ticker='IBM')
pd.concat([IBM,A])

stock_list=['IBM','A']
temp=pd.DataFrame()

for j in stock_list:
  T=Stocks(ticker=j)
  temp=pd.concat([temp,T])
temp


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,year,quarter,open,dividend,ticker
0,1999,4,102.560000,0.000000,IBM
1,2000,1,108.936667,0.040000,IBM
2,2000,2,113.353333,0.043333,IBM
3,2000,3,117.936667,0.043333,IBM
4,2000,4,101.833333,0.043333,IBM
...,...,...,...,...,...
86,2021,2,133.820000,0.064667,A
87,2021,3,158.953333,0.064667,A
88,2021,4,155.216667,0.064667,A
89,2022,1,143.453333,0.070000,A


In [13]:
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=IBM&apikey=6JKXZSE93BPMCPCY'
r = requests.get(url)
data = r.json()
data.keys()

earnings=pd.DataFrame.from_dict(data['quarterlyEarnings'])
earnings.rename(columns={'fiscalDateEnding':'date'},inplace=True)
earnings['date']= pd.to_datetime(earnings['date'])
earnings['month']=earnings['date'].dt.month
earnings['quarter']=earnings['month']/3
earnings.head()

,date,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,month,quarter
0,2022-03-31,2022-04-19,1.4,1.38,0.02,1.4493,3,1.0
1,2021-12-31,2022-01-24,3.35,3.3,0.05,1.5152,12,4.0
2,2021-09-30,2021-10-20,2.52,2.5147,0.0053,0.2108,9,3.0
3,2021-06-30,2021-07-19,2.33,2.2884,0.0416,1.8179,6,2.0
4,2021-03-31,2021-04-19,1.77,1.6524,0.1176,7.1169,3,1.0
